In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://sec

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-18 14:04:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.65MB/s    in 0.2s    

2021-08-18 14:04:43 (5.65 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# start sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LiveData").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
!wget https://drewholt12-joineddata.s3.us-east-2.amazonaws.com/database_join.csv
!touch "/content/database_join.csv"

--2021-08-18 14:04:56--  https://drewholt12-joineddata.s3.us-east-2.amazonaws.com/database_join.csv
Resolving drewholt12-joineddata.s3.us-east-2.amazonaws.com (drewholt12-joineddata.s3.us-east-2.amazonaws.com)... 52.219.97.202
Connecting to drewholt12-joineddata.s3.us-east-2.amazonaws.com (drewholt12-joineddata.s3.us-east-2.amazonaws.com)|52.219.97.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382066 (373K) [text/csv]
Saving to: ‘database_join.csv’

database_join.csv   100%[===================>] 373.11K  --.-KB/s    in 0.06s   

2021-08-18 14:04:56 (5.66 MB/s) - ‘database_join.csv’ saved [382066/382066]



In [5]:
# read the file and save locally
import re
from pyspark import SparkFiles

with open ("/content/database_join.csv","r+") as f:
  content = f.read()
  content_new = re.sub('(?<!")\n', ' ', content, flags=re.M)
  content_new = re.sub('Mapped Location', 'Mapped Location\n', content_new)
with open("/content/database_join.csv", "w") as w:
  w.write(content_new)

spark.sparkContext.addFile("/content/database_join.csv")
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show()

+-----+--------+----------------+---------------------+-----------------------+---------------+--------------------+----------------------+------------------------------------+--------------------------------------------------+----------------------------------------------+----------------------------------------------------+-------------------------------------+---------------------------------------------------+-----------------------------------------------+-----------------------------------------------------+-------------------------------------+---------------------------------------------------+-----------------------------------------------+-----------------------------------------------------+----------------------------+------------------------------------------+--------------------------------------+--------------------------------------------+------------------------+-------+-------------+--------------+------------+-----------+--------------------+----------+---------+----

In [7]:
# Initial imports.
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.multioutput import MultiOutputClassifier

In [8]:
# convert file to dataFrame
df = pd.read_csv("/content/database_join.csv")
df.head()

,index,zip_code,total_population,total_male_population,total_female_popularion,eight_teen_plus,eight_teen_plus_male,eight_teen_plus_female,population_eight_teen_to_twenty_four,eight_teen_to_twenty_four_not_high_school_graduate,eight_teen_to_twenty_four_high_school_graduate,eight_teen_to_twenty_four_bachelors_degree_or_higher,population_twenty_five_to_thirty_four,twenty_five_to_thirty_four_not_high_school_graduate,twenty_five_to_thirty_four_high_school_graduate,twenty_five_to_thirty_four_bachelors_degree_or_higher,population_thirty_five_to_fourty_four,thirty_five_to_fourty_four_not_high_school_graduate,thirty_five_to_fourty_four_high_school_graduate,thirty_five_to_fourty_four_bachelors_degree_or_higher,population_sixty_five_and_up,sixty_five_and_up_not_high_school_graduate,sixty_five_and_up_high_school_graduate,sixty_five_and_up_bachelors_degree_or_higher,average_household_income,index-2,permit_number,permit_subtype,date_entered,date_issued,business_name,zip_code-2,latitude,longitude,permit_duration
0,0,37013,97939,47658,50281,59479,27984,31495,9099,1073,6670,1356,19538,1439,10123,7976,14718,2146,7680,4892,6688,1045,4248,1395,26832,0,19-25846,ONSALES,2019-01-24,2019-06-03,EXO-TIKKA INDIAN CUSINE,37013,36.047649,-86.648119,797 days
1,18,37209,38664,18251,20413,29326,13363,15963,5522,337,3591,1594,11095,434,3399,7262,4738,744,2011,1983,3052,554,1801,697,32539,1,16-20340,ONSALES,2016-05-18,2016-09-23,Las Palmas Mexican Restaurant,37209,36.149076,-86.863105,1780 days
2,24,37215,22379,9989,12390,17874,7986,9888,1774,222,848,704,2717,37,434,2246,2488,24,378,2086,5135,121,1211,3803,88326,2,19-25920,ONOFFSALES,2019-03-06,2019-06-13,NASHVILLE CIGAR,37215,36.104329,-86.815039,787 days
3,15,37206,26382,12473,13909,20213,9555,10658,1990,228,1435,327,6918,226,2506,4186,4208,364,1408,2436,2226,450,1369,407,36851,3,19-25985,ONOFFSALES,2019-04-22,2019-06-13,TAILGATE BREWERY EAST NASHVILLE,37206,36.186415,-86.747210,787 days
4,18,37209,38664,18251,20413,29326,13363,15963,5522,337,3591,1594,11095,434,3399,7262,4738,744,2011,1983,3052,554,1801,697,32539,4,17-25076,ONSALES,2017-06-07,2018-03-02,BARE BONES BUTCHER,37209,36.158192,-86.848890,1255 days


In [9]:
# display all column names
df.columns.values.tolist()

['index',
 'zip_code',
 'total_population',
 'total_male_population',
 'total_female_popularion',
 'eight_teen_plus',
 'eight_teen_plus_male',
 'eight_teen_plus_female',
 'population_eight_teen_to_twenty_four',
 'eight_teen_to_twenty_four_not_high_school_graduate',
 'eight_teen_to_twenty_four_high_school_graduate',
 'eight_teen_to_twenty_four_bachelors_degree_or_higher',
 'population_twenty_five_to_thirty_four',
 'twenty_five_to_thirty_four_not_high_school_graduate',
 'twenty_five_to_thirty_four_high_school_graduate',
 'twenty_five_to_thirty_four_bachelors_degree_or_higher',
 'population_thirty_five_to_fourty_four',
 'thirty_five_to_fourty_four_not_high_school_graduate',
 'thirty_five_to_fourty_four_high_school_graduate',
 'thirty_five_to_fourty_four_bachelors_degree_or_higher',
 'population_sixty_five_and_up',
 'sixty_five_and_up_not_high_school_graduate',
 'sixty_five_and_up_high_school_graduate',
 'sixty_five_and_up_bachelors_degree_or_higher',
 'average_household_income',
 'index-2

In [10]:
# select columns needed for this model
df = df[["zip_code","total_male_population","total_female_popularion",
 "average_household_income","permit_number","permit_duration"]]
df.head()

,zip_code,total_male_population,total_female_popularion,average_household_income,permit_number,permit_duration
0,37013,47658,50281,26832,19-25846,797 days
1,37209,18251,20413,32539,16-20340,1780 days
2,37215,9989,12390,88326,19-25920,787 days
3,37206,12473,13909,36851,19-25985,787 days
4,37209,18251,20413,32539,17-25076,1255 days


In [11]:
# split permit duration to get days for calculation puposes
days = df['permit_duration'].str.split(' ', n=1, expand =True)
df["days"] = days[0].astype(int)


In [12]:
# define success as 2 years with permit, create new binary column to represent success vs failure
df["success"] = 0
df.loc[df["days"] >= 750, "success"] = 1

df.head()

,zip_code,total_male_population,total_female_popularion,average_household_income,permit_number,permit_duration,days,success
0,37013,47658,50281,26832,19-25846,797 days,797,1
1,37209,18251,20413,32539,16-20340,1780 days,1780,1
2,37215,9989,12390,88326,19-25920,787 days,787,1
3,37206,12473,13909,36851,19-25985,787 days,787,1
4,37209,18251,20413,32539,17-25076,1255 days,1255,1


In [13]:
# create DF to hold permit numbers because scaling must be done with numeric values
pn_df = pd.DataFrame(df['permit_number']).set_index(df.index)

In [14]:
# drop unnecessary columns
df.drop(columns=["permit_number", "permit_duration", "days"], inplace = True)

In [15]:
# Define the features set.
X = df.copy()
X = X.drop("success", axis=1)
y = df["success"]

In [16]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

In [19]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [20]:
# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [21]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [22]:
# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,1,88
Actual 1,1,236


In [23]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1,88
Actual 1,1,236


Accuracy Score : 0.7269938650306749
Classification Report
              precision    recall  f1-score   support

           0       0.50      0.01      0.02        89
           1       0.73      1.00      0.84       237

    accuracy                           0.73       326
   macro avg       0.61      0.50      0.43       326
weighted avg       0.67      0.73      0.62       326



In [24]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances

array([0.33739118, 0.21109754, 0.20146852, 0.25004275])

In [25]:
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3373911847849122, 'zip_code'),
 (0.2500427497829374, 'average_household_income'),
 (0.211097541832203, 'total_male_population'),
 (0.2014685235999475, 'total_female_popularion')]

In [26]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
   classifier = GradientBoostingClassifier(n_estimators=20,
   learning_rate=learning_rate,
   max_features=5,
   max_depth=3,
   random_state=0)
   classifier.fit(X_train_scaled, y_train.ravel)

TypeError: ignored